In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import stft 
from scipy.signal import medfilt as mediann
from librosa import stft, istft
import signals as get
from plotting import plot


## HPS: Separacion Armonica y Percusiva.
Este algoritmo busca seraparar las componetes armonicas y percusivas de una señal de audio, con el objetivo de implementar un TSM hibrido que procesa cada una de estas componentes por serapado,  el OLA y PV  aprovechando las mejores cualidades de cada uno.

La entrada de este algoritmo es la señal de audio con su correspondiente $F_s$, luego tenemos los parametros quen permite modificar son: 

-$N$: El largo de la STFT tiene una gran ifluencia en el desempeño del algoritmo existe una relacion de compromiso entre resolucion en frecuencia y tiempo. Cuanto mayor sea este valor mejora en frecuencia y pasa lo contrario en tiempo. 

-$M$: El largo de la ventana del fitro de mediana. A mayores valores aumenta el suavisado que genera. 

TERMINAR

A continuacion se detallan las pruebas de este algoritmo las cuales las dividimos en dos partes. Primero se muestran las objetivas, en las cules se busca medir los resultados con señales generadas digitalmente. Luego realizamos estas tipo de pruebas en contexto de TSM.
Y finalmente buscaremos evaluar al algoritmo de forma subjetiva con señales musicales para realizar un TSM. 

# Puebas Objetivas.

Primero generamos la señal de prueba $x[n]_e$ que sera la suma de otras señales puramente armonicas $x[n]_a$ y percusivas $x[n]$. En teoria si el aglgoritmo fuese perfecto, si ingresamos $x[n]_e$ deberia devolvernos a la salida $x[n]_a$ y $x[n]_p$ por lo tanto estas señales seran utiles para compararlas con la salida y medir eficacia del metodo. 

 